# **COVID-19’s Impact on the UAE’s Economy: Evidence of its reliance on policies of foreign countries:**

Note: If you have cloned the repository, there is normally no need to install anything externally. If this is not the case, please let me know. The only thing that will need to be changed on the readers end is to change the paths to fit where the repository is on the machine.

Link to Nighttime Lights Dataset: https://payneinstitute.mines.edu/eog/

## 1) Setting up: Importing necessary libraries and defining the winsorization function:

Note: One can find the documentation for these libraries online if needed, installing may be required if they have not been installed previously.

In [1]:
using Rasters
using Dates
using Statistics
using ArchGDAL
using Shapefile
using DataFrames

In [2]:
# Define the winsorization function
function winsorize_image(image, low_percentile::Float64, high_percentile::Float64)
    min_value = quantile(vec(image), low_percentile)
    max_value = quantile(vec(image), high_percentile)
    winsorized_image = clamp.(image, min_value, max_value)
    return winsorized_image
end

winsorize_image (generic function with 1 method)

## 2) Getting the Shapefile for the UAE:

This allows us to get the exact borders for the UAE in order to obtain accurate nighttime light intensities.

Link to the Shapefiles used: https://github.com/wri/wri-bounds/blob/master/README.md

In [3]:
path_shp = "/Users/nslei/OneDrive/Stanford_IHP/ECON109/Project/UAE/lockdown-uae/" # Change with your own path to the repository
shp = Shapefile.Table("$path_shp/all_primary_countries/all_primary_countries.shp") |> DataFrame

Row,geometry,ADM0_A3,abbrev,continent,formal_nam,iso_a2,iso_a3,iso_n3,iso_short,name,name_sort,postal,primary,short_name,sov_a3,sovereign,type,un_ar,un_en,un_es,un_fr,un_n3,un_region,un_ru,un_subregi,un_zh,wb_a2,wb_a3,wb_region
,Polygon?,String?,String?,String?,String?,String?,String?,Int64?,String?,String?,String?,String?,Int64?,String?,String?,String?,String?,String?,String?,String?,String?,Int64?,String?,String?,String?,String?,String?,String?,String?
1,Polygon(1530 Points),AFG,Afg.,Asia,Islamic State of Afghanistan,AF,AFG,4,Afghanistan,Afghanistan,Afghanistan,AF,1,Afghanistan,AFG,Afghanistan,Sovereign country,ناتسناغفأ,Afghanistan,Afganistán (el),Afghanistan (l') [masc.],4,Asia,Афганистан,Southern Asia,阿富汗,AF,AFG,South Asia
2,Polygon(1642 Points),AGO,Ang.,Africa,People's Republic of Angola,AO,AGO,24,Angola,Angola,Angola,AO,1,Angola,AGO,Angola,Sovereign country,الوغنأ,Angola,Angola,Angola (l') [masc.],24,Africa,Ангола,Middle Africa,安哥拉,AO,AGO,Sub-Saharan Africa
3,Polygon(554 Points),ALB,Alb.,Europe,Republic of Albania,AL,ALB,8,Albania,Albania,Albania,AL,1,Albania,ALB,Albania,Sovereign country,اينابلأ,Albania,Albania,Albanie (l') [fém.],8,Europe,Албания,Southern Europe,阿尔巴尼亚,AL,ALB,Europe & Central Asia
4,Polygon(51 Points),AND,And.,Europe,Principality of Andorra,AD,AND,20,Andorra,Andorra,Andorra,AND,1,Andorra,AND,Andorra,Sovereign country,ارودنأ,Andorra,Andorra,Andorre (l') [fém.],20,Europe,Андорра,Southern Europe,安道尔,AD,ADO,Europe & Central Asia
5,Polygon(796 Points),ARE,U.A.E.,Asia,United Arab Emirates,AE,ARE,784,United Arab Emirates (the),United Arab Emirates,United Arab Emirates,AE,1,United Arab Emirates,ARE,United Arab Emirates,Sovereign country,ةدحتملا ةيبرعلا تارامإلا,United Arab Emirates (the),Emiratos Árabes Unidos (los),Émirats arabes unis (les),784,Asia,Объединенные Арабские Эмираты,Western Asia,阿拉伯联合酋长国,AE,ARE,Middle East & North Africa
6,Polygon(4696 Points),ARG,Arg.,South America,Argentine Republic,AR,ARG,32,Argentina,Argentina,Argentina,AR,1,Argentina,ARG,Argentina,Sovereign country,نيتنجرألا,Argentina,Argentina (la),Argentine,32,Americas,Аргентина,South America,阿根廷,AR,ARG,Latin America & Caribbean
7,Polygon(433 Points),ARM,Arm.,Asia,Republic of Armenia,AM,ARM,51,Armenia,Armenia,Armenia,ARM,1,Armenia,ARM,Armenia,Sovereign country,اينيمرأ,Armenia,Armenia,Arménie (l') [fém.],51,Asia,Армения,Western Asia,亚美尼亚,AM,ARM,Europe & Central Asia
8,Polygon(108 Points),ATG,Ant.B.,North America,Antigua and Barbuda,AG,ATG,28,Antigua and Barbuda,Antigua and Barbuda,Antigua and Barbuda,AG,1,Antigua and Barb.,ATG,Antigua and Barbuda,Sovereign country,ادوبربو اوغيتنأ,Antigua and Barbuda,Antigua y Barbuda,Antigua-et-Barbuda [fém.],28,Americas,Антигуа и Барбуда,Caribbean,安提瓜和巴布达,AG,ATG,Latin America & Caribbean
9,Polygon(13650 Points),AUS,Auz.,Oceania,Commonwealth of Australia,AU,AUS,36,Australia,Australia,Australia,AU,1,Australia,AU1,Australia,Sovereign country,ايلارتسأ,Australia,Australia,Australie (l') [fém.],36,Oceania,Австралия,Australia and New Zealand,澳大利亚,AU,AUS,East Asia & Pacific


In [ ]:
uae_index = findall( x -> x == "United Arab Emirates", shp.name) # This gives us the shapefile for the UAE
uae_shape = shp.geometry[uae_index[1]]

Polygon(796 Points)

## 3) Producing the Radiance Datacube:

Link to the documentation for the radiance Datacube structure: https://xkdr.github.io/NighttimeLights.jl/dev/.

Link to the bounding box used: https://xkdr.github.io/NighttimeLights.jl/dev/. Note that this was slightly adjusted in the code.

In [ ]:
path = "/Users/nslei/OneDrive/Stanford_IHP/ECON109/Project/UAE/lockdown-uae/NTL_data" # Insert your path to the repository here.
filelist = readdir(path)
dates = [
    Date(match(r"\d{8}", f).match, dateformat"yyyymmdd")
    for f in filelist
]
bounds = X(Rasters.Between(51.58, 56.38)), Y(Rasters.Between(22.63, 26.05))   


(↓ X Between((51.58, 56.38)), → Y Between((22.63, 26.05)))

In [7]:
radiance_raster_list = [
    Raster(joinpath(path, f), lazy=true)[bounds...] 
    for f in filelist
]

14-element Vector{Raster{Float32, 2, Tuple{X{Projected{Float64, LinRange{Float64, Int64}, DimensionalData.Dimensions.Lookups.ForwardOrdered, DimensionalData.Dimensions.Lookups.Regular{Float64}, DimensionalData.Dimensions.Lookups.Intervals{DimensionalData.Dimensions.Lookups.Start}, DimensionalData.Dimensions.Lookups.NoMetadata, WellKnownText{GeoFormatTypes.CRS}, Nothing, X{Colon}}}, Y{Projected{Float64, LinRange{Float64, Int64}, DimensionalData.Dimensions.Lookups.ReverseOrdered, DimensionalData.Dimensions.Lookups.Regular{Float64}, DimensionalData.Dimensions.Lookups.Intervals{DimensionalData.Dimensions.Lookups.Start}, DimensionalData.Dimensions.Lookups.NoMetadata, WellKnownText{GeoFormatTypes.CRS}, Nothing, Y{Colon}}}}, Tuple{Band{DimensionalData.Dimensions.Lookups.Categorical{Int64, UnitRange{Int64}, DimensionalData.Dimensions.Lookups.ForwardOrdered, DimensionalData.Dimensions.Lookups.NoMetadata}}}, Matrix{Float32}, Symbol, DimensionalData.Dimensions.Lookups.Metadata{Rasters.GDALsource,

In [8]:
radiance_series = RasterSeries(radiance_raster_list, Ti(dates))
radiance_datacube = Rasters.combine(radiance_series, Ti)[bounds...]

╭───────────────────────────────╮
│ 1151×820×14 Raster{Float32,3} │
├───────────────────────────────┴──────────────────────────────────────── dims ┐
  ↓ X  Projected{Float64} LinRange{Float64}(51.581250892650004, 56.372917597649995, 1151) ForwardOrdered Regular Intervals{Start},
  → Y  Projected{Float64} LinRange{Float64}(26.04374960834999, 22.631249581049996, 820) ReverseOrdered Regular Intervals{Start},
  ↗ Ti Sampled{Date} [2019-03-01, …, 2023-10-01] ForwardOrdered Irregular Points
├──────────────────────────────────────────────────────────────────── metadata ┤
  Metadata{Rasters.GDALsource} of Dict{String, Any} with 4 entries:
  "units"    => ""
  "offset"   => 0.0
  "filepath" => "/Users/nslei/OneDrive/Stanford_IHP/ECON109/Project/UAE/lockdow…
  "scale"    => 1.0
├────────────────────────────────────────────────────────────────────── raster ┤
  extent: Extent(X = (51.581250892650004, 56.377084264349996), Y = (22.631249581049996, 26.047916275049992), Ti = (Date("2019-03-01"), Date(

In [9]:
total_light_per_month = []

for i in 1:length(dates)
    raster_uae = Raster(view(radiance_datacube, Ti(i)))
    raster_cropped = crop(raster_uae; to = shp.geometry[uae_index])

    raster_masked = mask(raster_cropped; with = shp.geometry[uae_index])
    arr = Array(raster_masked)

    arr_nonmissing = collect(skipmissing(arr))
    arr_winsorized = winsorize_image(arr_nonmissing, 0.01, 0.99)

    total_sum = sum(skipmissing(arr_winsorized))
    push!(total_light_per_month, total_sum)
end

In [10]:
uae_light_sums = Dict(dates[i] => total_light_per_month[i] for i in 1:length(dates))

Dict{Date, Float64} with 14 entries:
  Date("2020-03-01") => 1.56824e6
  Date("2023-10-01") => 1.89247e6
  Date("2022-09-01") => 1.81791e6
  Date("2019-03-01") => 1.55143e6
  Date("2020-05-01") => 1.57293e6
  Date("2020-08-01") => 1.52345e6
  Date("2019-06-01") => 1.58671e6
  Date("2019-05-01") => 1.63234e6
  Date("2020-04-01") => 1.64561e6
  Date("2020-06-01") => 1.53042e6
  Date("2021-08-01") => 1.64686e6
  Date("2020-02-01") => 1.61187e6
  Date("2020-09-01") => 1.62196e6
  Date("2020-07-01") => 1.54918e6

## 4) Printing the results

In [11]:
for (month, total) in sort(uae_light_sums)
    println("$month → Winsorized total radiance: $total")
end

2019-03-01 → Winsorized total radiance: 1.5514326499896795e6
2019-05-01 → Winsorized total radiance: 1.6323424356693774e6
2019-06-01 → Winsorized total radiance: 1.5867051288760006e6
2020-02-01 → Winsorized total radiance: 1.6118737161283654e6
2020-03-01 → Winsorized total radiance: 1.5682395723239481e6
2020-04-01 → Winsorized total radiance: 1.6456131177985081e6
2020-05-01 → Winsorized total radiance: 1.5729348255057633e6
2020-06-01 → Winsorized total radiance: 1.5304188946522183e6
2020-07-01 → Winsorized total radiance: 1.5491843054264453e6
2020-08-01 → Winsorized total radiance: 1.523449965577513e6
2020-09-01 → Winsorized total radiance: 1.6219561419276e6
2021-08-01 → Winsorized total radiance: 1.6468595728129565e6
2022-09-01 → Winsorized total radiance: 1.8179142553801537e6
2023-10-01 → Winsorized total radiance: 1.8924718623865095e6


In [18]:
# Function to retrieve the data between two tim periods:
function get_range_indices(dates, start_date::Date, end_date::Date)
    findall(d -> d >= start_date && d <= end_date, dates)
end


# First period: March 2019 to February 2020
idx1 = get_range_indices(dates, Date(2019,3,1), Date(2020,2,29))
vals1 = total_light_per_month[idx1]
slope1 = (last(vals1) - first(vals1)) / 11              # 11 months in this period
log_slope1 = (log(last(vals1)) - log(first(vals1))) / 11
println("Slope (Mar 2019 – Feb 2020): ", slope1)
println("Log-Slope (Mar 2019 – Feb 2020): ", log_slope1)
println()

# Second period: September 2020 to August 2021
idx2 = get_range_indices(dates, Date(2020,9,1), Date(2021,8,31))
vals2 = total_light_per_month[idx2]
slope2 = (last(vals2) - first(vals2)) / 11
log_slope2 = (log(last(vals2)) - log(first(vals2))) / 11
println("Slope (Sep 2020 – Aug 2021): ", slope2)
println("Log-Slope (Sep 2020 – Aug 2021): ", log_slope2)
println()

# Third period: August 2021 to September 2022
idx3 = get_range_indices(dates, Date(2021,8,1), Date(2022,9,30))
vals3 = total_light_per_month[idx3]
slope3 = (last(vals3) - first(vals3)) / 13              # 13 months in this period
log_slope3 = (log(last(vals3)) - log(first(vals3))) / 13 
println("Slope (Aug 2021 – Sep 2022): ", slope3)
println("Log-Slope (Aug 2021 – Sep 2022): ", log_slope3)
println()

# Fourth period: September 2022 to October 2023
idx4 = get_range_indices(dates, Date(2022,9,1), Date(2023,10,31))
vals4 = total_light_per_month[idx4]
slope4 = (last(vals4) - first(vals4)) / 13
log_slope4 = (log(last(vals4)) - log(first(vals4))) / 13
println("Slope (Sep 2022 – Oct 2023): ", slope4)
println("Log-Slope (Sep 2022 – Oct 2023): ", log_slope4)

Slope (Mar 2019 – Feb 2020): 5494.642376244174
Log-Slope (Mar 2019 – Feb 2020): 0.00347440970828685

Slope (Sep 2020 – Aug 2021): 2263.948262305147
Log-Slope (Sep 2020 – Aug 2021): 0.0013852063023871027

Slope (Aug 2021 – Sep 2022): 13158.05250516901
Log-Slope (Aug 2021 – Sep 2022): 0.007601511171835922

Slope (Sep 2022 – Oct 2023): 5735.200538950453
Log-Slope (Sep 2022 – Oct 2023): 0.003091846759514529
